<a href="https://colab.research.google.com/github/august3sas/OOP-put-course/blob/main/Data_Mining_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Mining - Project

Jupyter notebook for data mining project.

## Predicting property prices using location attributes & characteristics

### 0. Pulling the datasets

In [12]:
!pip install gdown -q
!sudo apt-get install tree -q

import os
from pathlib import Path

data_dir = Path('data')
data_dir.mkdir(exist_ok=True)

folder_url = 'https://drive.google.com/drive/folders/1vgj_pa2kw-vlT5XmC_Yft7cJAASd_xh7'

!gdown --folder {folder_url} -O {data_dir}

print("\nDownloaded files:")
!tree {data_dir}


Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 29 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 0s (124 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package tree.
(Reading database ... 126209 files and directories currently installed.)
Preparing 

In [13]:
import pandas as pd

In [20]:
chp = pd.read_csv('data/california-housing-prices.csv')
hd = pd.read_csv('data/housedata.csv')
melb = pd.read_csv('data/melbourne-housing-full.csv')
nyc = pd.read_csv('data/nyc-rolling-sales.csv')
red = pd.read_csv('data/real-estate-dataset.csv')
repp = pd.read_csv('data/real-estate-price-prediction.csv')
ured = pd.read_csv('data/usa-real-estate-dataset.csv')

### 1. Data Exploration

Let's take a glance at data that is available in the datasets

In [34]:
def print_dataset_overview(dataset, dataset_name):
  print()
  print('============================')
  print(dataset_name)
  print('============================')
  print('Attributes')
  print(dataset.columns)
  print()
  print('Number of rows')
  print(dataset.shape[0])
  print()
  print('Number of columns')
  print(dataset.shape[1])
  print()
  print('First 5 rows')
  print(dataset.head())
  print()
  print('Info')
  print(dataset.info())
  print()
  print('Describe')
  print(dataset.describe())

In [24]:
print_dataset_overview(ured, 'USA Real Estate Dataset')
print_dataset_overview(chp, 'California Housing Prices')
print_dataset_overview(hd, 'House Data')
print_dataset_overview(melb, 'Melbourne Housing Full')
print_dataset_overview(nyc, 'NYC Rolling Sales')
print_dataset_overview(repp, 'Real Estate Price Prediction')


USA Real Estate Dataset
Attributes
Index(['brokered_by', 'status', 'price', 'bed', 'bath', 'acre_lot', 'street',
       'city', 'state', 'zip_code', 'house_size', 'prev_sold_date'],
      dtype='object')

Number of rows
2226382

Number of columns
12

First 5 rows
   brokered_by    status     price  bed  bath  acre_lot     street  \
0     103378.0  for_sale  105000.0  3.0   2.0      0.12  1962661.0   
1      52707.0  for_sale   80000.0  4.0   2.0      0.08  1902874.0   
2     103379.0  for_sale   67000.0  2.0   1.0      0.15  1404990.0   
3      31239.0  for_sale  145000.0  4.0   2.0      0.10  1947675.0   
4      34632.0  for_sale   65000.0  6.0   2.0      0.05   331151.0   

         city        state  zip_code  house_size prev_sold_date  
0    Adjuntas  Puerto Rico     601.0       920.0            NaN  
1    Adjuntas  Puerto Rico     601.0      1527.0            NaN  
2  Juana Diaz  Puerto Rico     795.0       748.0            NaN  
3       Ponce  Puerto Rico     731.0      1800.0  

#### Attribute selection

From the usa-real-estate-dataset.csv we can see that most likey, 'brokered_by' and 'status' attributes will be unsignificant and redundant, since it encodes the agency that sold the property and indicates the sale status, which is not the aim of this analysis. Even though 'prev_sold_date' also describes the sale of the property, we will leave it be, since it indicates, if the property was sold before.

Also, the 'street' attribute is way too specific. For determining the location, 'state','city' and 'zip_code' should be sufficient.

In summary, the attributes we selected for exploratory analysis are:
- price (the attribute we are trying to predict)
- bed (the number of bedrooms)
- bath (the number of bathrooms)
- acre_lot (total lot size)
- city
- state
- zip_code
- house_size
- prev_sold_date

In [29]:
ured_filtered = ured.drop(columns=['brokered_by', 'street', 'status'])

In [30]:
print_dataset_overview(ured_filtered, 'USA Real Estate Dataset - filtered')


USA Real Estate Dataset - filtered
Attributes
Index(['price', 'bed', 'bath', 'acre_lot', 'city', 'state', 'zip_code',
       'house_size', 'prev_sold_date'],
      dtype='object')

Number of rows
2226382

Number of columns
9

First 5 rows
      price  bed  bath  acre_lot        city        state  zip_code  \
0  105000.0  3.0   2.0      0.12    Adjuntas  Puerto Rico     601.0   
1   80000.0  4.0   2.0      0.08    Adjuntas  Puerto Rico     601.0   
2   67000.0  2.0   1.0      0.15  Juana Diaz  Puerto Rico     795.0   
3  145000.0  4.0   2.0      0.10       Ponce  Puerto Rico     731.0   
4   65000.0  6.0   2.0      0.05    Mayaguez  Puerto Rico     680.0   

   house_size prev_sold_date  
0       920.0            NaN  
1      1527.0            NaN  
2       748.0            NaN  
3      1800.0            NaN  
4         NaN            NaN  

Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2226382 entries, 0 to 2226381
Data columns (total 9 columns):
 #   Column          Dtype  


#### Clearing empty values

Firstly, let's inspect the columns for empty values

In [31]:
def count_nans(dataset):
  for column in dataset.columns:
    print(f'{column}: {dataset[column].isna().sum()}')

In [33]:
count_nans(ured_filtered)
count_nans(chp)

price: 1541
bed: 481317
bath: 511771
acre_lot: 325589
city: 1407
state: 8
zip_code: 299
house_size: 568484
prev_sold_date: 734297
longitude: 0
latitude: 0
housing_median_age: 0
total_rooms: 0
total_bedrooms: 207
population: 0
households: 0
median_income: 0
median_house_value: 0
ocean_proximity: 0


Handling NaN values:

- in 'price' - entries will be dropped
- in 'bed' and 'bath' - ...
- in 'acre_lot' - ...
- in 'city', 'state', 'zip' - entries will be dropped
- in 'house_size' - ...

- for 'prev_sold_date' we will introduce a new variable, that indicates, if the property has been sold before, since it could affect the final price.